In [8]:
import yaml
import pprint

slot_dict = yaml.safe_load(open("intent_slots.yml", 'r'))
pprint.pprint(slot_dict)

{'Intent-Slot': {'Bye': [],
                 'Crop Quality Query': ['Crop Type',
                                        'User Location',
                                        'Crop Symptoms',
                                        'Solution'],
                 'Greeting': [],
                 'Machinery Query': ['Machine Symptoms', 'Solution'],
                 'Seed Recommendation Query': ['Crop Type', 'Solution'],
                 'UNK': [],
                 'Yielding Season Query': ['Crop Type',
                                           'User Location',
                                           'Solution']},
 'Slot-Prompt': {'Crop Symptoms': 'ap ke fasal ko kia howa hai',
                 'Crop Type': 'ap kon se fasal auga rahe hain',
                 'Machine Symptoms': 'ap ke machine ko kia howa hai',
                 'Solution': 'phir yeh hai ap ka hal',
                 'User Location': 'ap kidhar kaam kar rahe hain'}}


In [9]:
class KisaanDost:
    def __init__(self):
        self.chat_history = []
        self.problems = {'Solved':[],'Unsolved':[]}
        self.intent_classifier = None
        self.entity_extractor = None
        self.knowledge_base = None
    def query_knowledge_base(self,query_info):
        pass
    def get_solution(self,problem):
        return slot_dict['Slot-Prompt']['Solution']
    def classify_intent(self,user_input_dict):
        """
        Classifies message into one or more of the following intents.
            Greeting
            Bye
            UNK
            Machinery Query
            Yielding Season Query
            Seed Recommendation Query
            Crop Quality Query
        If no intent is detected in the message, UNK intent is classified. This is the only
        situation in which UNK intent is classified. UNK stands for 'unknown'.
        """
        user_input_dict['Intent'] = ["UNK"]        
    def extract_entities(self,user_input_dict):
        user_input_dict['Entities'] = []
        for problem in self.problems['Unsolved']:
            for slot_key in problem['Slots'].keys():
                if slot_key in user_input_dict['Entities']:
                    problem['Slots'][slot_key] = user_input_dict['Entities'][slot_key]
    def detect_problem(self,user_input_dict):
        for i in user_input_dict['Intent']:
            if "Query" in i:
                problem = i[:i.index('Query')-1]
                slots = {k:None for k in slot_dict['Intent-Slot'][i]}
                self.problems['Unsolved'].append({'Problem':problem,
                                                  'Slots':slots,
                                                  'Solution':None})
    def get_user_input(self,x=None):
        if not x:
            user_input = input()
        self.chat_history.append({'Agent':'User',
                                  'Message':user_input,
                                  'Intent':["UNK"],
                                  'Entities':[]})
    def generate_response(self,user_input_dict=None):
        try:
            if not user_input_dict:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'assalam alaikum'})
                return
            if 'Bye' in user_input_dict['Intent']:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'allah hafiz'})
                return
            if 'UNK' in user_input_dict['Intent']:
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':'meje ap ke baat samajh nhai ai'})
                return
            message = ''  
            if 'Greeting' in user_input_dict['Intent']:
                message += 'walaikum assalam '
                if len(user_input_dict['Intent'])==1:
                    message += 'ap ka kia masla hai'
                self.chat_history.append({'Agent':'KisaanBot',
                                          'Message':message})  
                return
            current_problem = self.problems['Unsolved'][0]
            all_slots_filled = True
            for slot in current_problem['Slots']:
                if not current_problem[slot]:
                    message += slot_dict['Slot-Prompt'][slot]
                    all_slots_filled = False
                    break
            if all_slots_filled:
                solution = get_solution(current_problem)
                message += solution
                current_problem['Solution'] = solution
                problem_index = self.problems['Unsolved'].index(current_problem)
                solved_problem = self.problems['Unsolved'].pop(problem_index)
                self.problems['Solved'].append(solved_problem)
            self.chat_history.append({'Agent':'KisaanBot',
                                      'Message':message})        
        except:
            self.chat_history.append({'Agent':'KisaanBot',
                                      'Message':'mein abhi chal nhai raha baad mein koshish kejiye ga'}) 
    def get_user_voice_input(self,x=None):
        pass
    def generate_voice_response(self):
        pass
    def return_last_response_message(self):
        for c in self.chat_history[::-1]:
            if c['Agent']=='KisaanBot':
                return c['Message']
    def run_chatbot(self):
        while(1):
            self.get_user_input()
            if self.chat_history[-1]['Message'] == 'q':
                break                
            self.classify_intent(self.chat_history[-1])
            self.extract_entities(self.chat_history[-1])
            self.detect_problem(self.chat_history[-1])
            self.generate_response(self.chat_history[-1])
            print(f"{self.chat_history[-1]['Agent']}: {self.chat_history[-1]['Message']}")

In [10]:
myKisaanDost = KisaanDost()
myKisaanDost.run_chatbot()

hmmm
KisaanBot: meje ap ke baat samajh nhai ai
q
